<center><h1>Multivariate K-Nearest Neighbors</h1></center>

## 1. Recap.
### Exercise
## 2. Removing features.
### Exercise
## 3. Handling missing values.
### Exercise
## 4. Normalize columns.
### Exercise
## 5. Euclidean distance for multivariate case.
### Exercise
## 6. Introduction to scikit-learn.
### Exercise
## 7. Fitting a model and making predictions.
### Exercise
## 8. Calculating MSE using Scikit-Learn.
### Exercise
## 9. Using more features.
### Exercise
## 10. Using all features.
### Exercise
## 11. Next steps.
### Exercise
## 12. Takeaways.
### Exercise